In [34]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.seattle.gov", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.seattle.gov,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
query = """
select *
where
    sourceelementkey = 1001 	
    and occupancydatetime between '2021-10-27T08:00:00' and '2021-10-27T19:00:00'
"""
results = client.get("jb6y-98nr", query=query)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)


In [36]:
results_df.head(1000)

,occupancydatetime,paidoccupancy,blockfacename,sideofstreet,sourceelementkey,parkingtimelimitcategory,parkingspacecount,paidparkingarea,paidparkingsubarea,parkingcategory,location
0,2021-10-27T08:00:00.000,0,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,SW,1001,120,4,Pioneer Square,Core,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3346935..."
1,2021-10-27T08:01:00.000,0,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,SW,1001,120,4,Pioneer Square,Core,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3346935..."
2,2021-10-27T08:02:00.000,0,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,SW,1001,120,4,Pioneer Square,Core,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3346935..."
3,2021-10-27T08:03:00.000,0,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,SW,1001,120,4,Pioneer Square,Core,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3346935..."
4,2021-10-27T08:04:00.000,0,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,SW,1001,120,4,Pioneer Square,Core,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3346935..."
...,...,...,...,...,...,...,...,...,...,...,...
595,2021-10-27T17:55:00.000,4,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,SW,1001,120,4,Pioneer Square,Core,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3346935..."
596,2021-10-27T17:56:00.000,4,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,SW,1001,120,4,Pioneer Square,Core,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3346935..."
597,2021-10-27T17:57:00.000,4,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,SW,1001,120,4,Pioneer Square,Core,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3346935..."
598,2021-10-27T17:58:00.000,4,1ST AVE BETWEEN CHERRY ST AND COLUMBIA ST,SW,1001,120,4,Pioneer Square,Core,Paid Parking,"{'type': 'Point', 'coordinates': [-122.3346935..."
